In [ ]:
import math
from heapq import heapify, heappush, heappop
import copy, math, sys, random

In [ ]:
fc = 2.4 #frequency
noise = -1743
pc, pd = [45, 24] # power of cellular user and devices
min_sinr = 0
dd_pairs = []
number_of_cu = 0

In [ ]:
 #g calculation
def calculate_gain(ob1, ob2):
    dis = math.sqrt((ob1[0]-ob2[0])**2 + (ob1[1]-ob2[1])**2)
    if(dis <= 0):
        return 22.7 + 26*math.log(fc)
    return 36.7*math.log(dis, 10) + 22.7 + 26*math.log(fc)

In [ ]:
def get_interf(object, dr): #interferece from each object in a chanel
    value = object["power"] * calculate_gain(object["cordinates"], dr)
    return value

In [ ]:
def calculate_intereference(ch, dr): #total intereference from a chanel
    sum = 0
    for object in ch: 
        sum = sum + get_interf(object, dr)
    return sum

In [ ]:
def get_sinr(ch, dt, dr):  #SINR value of pair with a chanel
    gain_dd = calculate_gain(dt, dr); 
    interference_value = calculate_intereference(ch, dr)
    sinr_value = (pd*gain_dd) / (noise+interference_value)
    return sinr_value

In [ ]:
def fitness_sphere(position):
    # print(type(position[1]), "\t", position)
    for i in range(len(position)):
        position[i] = int(position[i])

    fitness_value = 0.0
    chnls = []
    for i in range(number_of_cu):
        lst = []
        chnls.append(lst)

    for i in range(len(position)):
        DT = dd_pairs[i][:2]
        j = (position[i] - 1)
        # print(j)
        chnls[j].append({"power": pd, "cordinates": DT, "name" : "D"+str(i+1)})
    
    for i in range(len(position)):
        DT = dd_pairs[i][:2]
        DR = dd_pairs[i][2:]
        fitness_value += get_sinr(chnls[position[i]-1], DT, DR)

    return float(1.0/fitness_value)

In [ ]:

# -------------------------
# solution class
class solution:
    def __init__(self, fitness, dd_channel):
        self.position = dd_channel
        self.fitness = fitness(dd_channel)

In [ ]:


# bTeaching learning based optimization
def tlbo(fitness, max_iter, n, dim, dd_channel, minx, maxx):
    rnd = random.Random(0)

    population = [solution(fitness, dd_channel) for i in range(n)]

    # compute the value of best_position and best_fitness in the population
    Xbest = [0.0 for i in range(dim)]
    Fbest = sys.float_info.max

    for i in range(n):  # check each solution
        if population[i].fitness < Fbest:
            Fbest = population[i].fitness
            Xbest = copy.copy(population[i].position)

            # main loop of tlbo
            Iter = 0
            while Iter < max_iter:
                # print(Iter,max_iter)
                # after every 10 iterations
                # print iteration number and best fitness value so far
                if Iter % 20 == 0 and Iter > 1:
                    print("Iter = " + str(Iter) + " best fitness = %.3f" % Fbest)

                # for each solution of population
                for i in range(n):

                    ### Teaching phase of ith solution

                    # compute the mean of all the solution in the class
                    Xmean = [0.0 for i in range(dim)]
                    for k in range(n):
                        for j in range(dim):
                            Xmean[j] += population[k].position[j]

                    for j in range(dim):
                        Xmean[j] /= n

                    # initialize new solution
                    Xnew = [0.0 for i in range(dim)]

                    # teaching factor (TF)
                    # either 1 or 2 ( randomly chosen)
                    TF = random.randint(1, 2)

                    # best solution of the class is teacher
                    Xteacher = Xbest
                    # compute new solution
                    for j in range(dim):
                        Xnew[j] = population[i].position[j] + rnd.random() * (Xteacher[j] - TF * Xmean[j])

                    # if Xnew < minx OR Xnew > maxx
                    # then clip it
                    for j in range(dim):
                        Xnew[j] = max(Xnew[j], minx)
                        Xnew[j] = min(Xnew[j], maxx)

                    # compute fitness of new solution
                    fnew = fitness(Xnew)

                    # if new solution is better than old
                    # replace old with new solution
                    if (fnew < population[i].fitness):
                        population[i].position = Xnew
                        population[i].fitness = fnew

                        # update best student
                    if (fnew < Fbest):
                        Fbest = fnew
                        Xbest = Xnew

                     ### learning phase of ith solution

                        # randomly choose a solution from population
                        # chosen solution should not be ith solution
                        p = random.randint(0, n - 1)
                        while (p == i):
                            p = random.randint(0, n - 1)

                        # partner solution
                        Xpartner = population[p]

                        Xnew = [0.0 for i in range(dim)]
                        if (population[i].fitness < Xpartner.fitness):
                            for j in range(dim):
                                Xnew[j] = population[i].position[j] + rnd.random() * (population[i].position[j] - Xpartner.position[j])
                        else:
                            for j in range(dim):
                                Xnew[j] = population[i].position[j] - rnd.random() * (population[i].position[j] - Xpartner.position[j])

                        # if Xnew < minx OR Xnew > maxx
                        # then clip it
                        for j in range(dim):
                            Xnew[j] = max(Xnew[j], minx)
                            Xnew[j] = min(Xnew[j], maxx)

                        # compute fitness of new solution
                        fnew = fitness(Xnew)

                        # if new solution is better than old
                        # replace old with new solution
                        if (fnew < population[i].fitness):
                            population[i].position = Xnew
                            population[i].fitness = fnew

                        # update best solution
                        if (fnew < Fbest):
                            Fbest = fnew
                            Xbest = Xnew

                Iter += 1
            # return best solution from population
            return Xbest


In [ ]:
if __name__ == '__main__' : 
    BS = [50, 50]
    #BS = list(map(int,input("\n Base station cordinates of centre hexagon: ").strip().split()))[:]

    number_of_cu = 14
    #number_of_cu = int(input("\n number_of_cellularUsers: ",5))
    #print(" enter their cordinates: ")
    #cu = []
    #for i in range(0, number_of_cu):
        #ele = list(map(int,input().strip().split()))[:]
        #cu.append(ele)

    cu = [[5, 5],[10, 10], [15, 15], [20, 20], [25, 25], [30, 30], [35, 35], [40, 40], [45, 45], [50, 50],[30, 30], [35, 35], [40, 40], [45, 45]]
    print("\nCordinates of Cellualr Users: ")
    for user in cu:
        print(user)

    #number_of_dd = int(input("\n number of d2d pairs: "))
    #print(" enter their cordinates: ")
    #dd_pairs=[]
    #for i in range(0, number_of_dd):
        #ele = list(map(int,input().strip().split()))[:]
        #dd_pairs.append(ele)
    number_of_dd= 20
    print("\nnumber of D2D pairs: ", 20)

    dd_pairs = [[40, 50, 60, 70],
                [80, 60, 40, 20],
                [10, 20, 30, 40],
                [40, 30, 20, 10],
                [20, 10, 30, 10],
                [10, 20, 30, 40],
                [50, 60, 70, 80],
                [90, 70, 50, 30],
                [20, 40, 60, 80],
                [50, 70, 30, 20],
                [20, 30, 40, 50],
                [85, 56, 60, 71],
                [90, 80, 70, 60],
                [10, 20, 40, 60],
                [85, 75, 65, 55],
                [40, 50, 60, 70],
                [80, 60, 40, 20],
                [10, 20, 30, 40],
                [40, 30, 20, 10],
                [20, 10, 30, 10]]
               
    
    print("\nD2D pair cordinates: ")
    for pair in dd_pairs:
        print(pair)
    ch = []
    for i in range(number_of_cu):
        lst = []
        ch.append(lst)

    number_of_channels = number_of_cu

    heap = []
    heapify(heap)

    for i in range(number_of_cu):
        ch[i].append({"power":pc, "cordinates": cu[i], "name": "CU"+str(i+1)})
        heappush(heap, [1, i])

    arr = [0]*number_of_dd

    for i in range(number_of_dd):
        DT = dd_pairs[i][:2]
        DR = dd_pairs[i][2:]

        # print("heap = ", heap)

        while len(heap) > 0:
        # for chnl in heap:
            chnl = heap[0]
            count = chnl[0]
            j = chnl[1]
            heappop(heap)
            sinr = get_sinr(ch[j], DT, DR)


            if(sinr >= min_sinr):
                arr[i] = (j+1)
                ch[j].append({"power": pd, "cordinates": DT, "name" : "D"+str(i+1)})
                heappush(heap, [count+1, j])
                break 

    
    dd_channel = []

    for val in arr:
        dd_channel.append(int(val))

    print("\n channel allocated : ", dd_channel, "\n")

    fitness = fitness_sphere
    max_iter = 100
    num_particles = 10
    dim = number_of_dd

    best_allocation = tlbo(fitness, max_iter, num_particles, dim, dd_channel, 1, number_of_cu)
    
    print("\n Best allocation : ", [x  for x in best_allocation])
    err = fitness(best_allocation)
    print("fitness of best solution = %.6f" % err)
    print("\nEnd b-tlbo for sphere\n")


Cordinates of Cellualr Users: 
[5, 5]
[10, 10]
[15, 15]
[20, 20]
[25, 25]
[30, 30]
[35, 35]
[40, 40]
[45, 45]
[50, 50]
[30, 30]
[35, 35]
[40, 40]
[45, 45]

number of D2D pairs:  20

D2D pair cordinates: 
[40, 50, 60, 70]
[80, 60, 40, 20]
[10, 20, 30, 40]
[40, 30, 20, 10]
[20, 10, 30, 10]
[10, 20, 30, 40]
[50, 60, 70, 80]
[90, 70, 50, 30]
[20, 40, 60, 80]
[50, 70, 30, 20]
[20, 30, 40, 50]
[85, 56, 60, 71]
[90, 80, 70, 60]
[10, 20, 40, 60]
[85, 75, 65, 55]
[40, 50, 60, 70]
[80, 60, 40, 20]
[10, 20, 30, 40]
[40, 30, 20, 10]
[20, 10, 30, 10]

 channel allocated :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6] 

Iter = 20 best fitness = 0.027
Iter = 40 best fitness = 0.027
Iter = 60 best fitness = 0.027
Iter = 80 best fitness = 0.027

 Best allocation :  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 2, 3, 4, 5, 6]
fitness of best solution = 0.026834

End b-tlbo for sphere



In [ ]:
system_throughPut = 10* math.log2(1 + 1/fitness_sphere(dd_channel))
system_throughPut /= 1000000
print("throughput = ", system_throughPut)

throughput =  5.257992467406754e-05
